In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import csv
import ast
import math
import random
from scipy import stats

In [2]:
#import my data
files = os.listdir('./testTracks')
#nFiles = len(files)
nFiles = 200
#for i in range(0,len(files)):
for i in range(0,nFiles):
    files[i] = "testTracks/" + files[i]
    #print(files[i])
def phiUncertainty(hit):
    x = hit[0]
    y = hit[1]
    dx = hit[4]
    dy = hit[4]
    partialx = x/(x**2 + y**2)
    partialy = -x/(x**2 + y**2)
    dphi = np.sqrt((partialx*dx)**2 + (partialy*dy)**2)
    return(dphi)

FileNotFoundError: [Errno 2] No such file or directory: './testTracks'

In [ ]:
#store the data as an array of tracks
tracks = []
hitList = []
maxhits = 0
#read each file
for i in range(0,nFiles):
    with open(files[i],'r') as f:
        lines = f.readlines()
        lines.pop(0)
        #array to store the hits that make up the track
        hits = []
        
        for i in range(0,len(lines)):
            lines[i] = lines[i].rstrip()
            hit = [np.float64(x) for x in lines[i].split(' ')]
            #print(hit)
            #calculate phi information
            phi = np.arctan(hit[1]/hit[0])
            r = np.sqrt(hit[0]**2 + hit[1]**2)
            dphi = phiUncertainty(hit)
            x = hit[0]
            y = hit[1]
            hits.append([r,phi,hit[2],hit[3],hit[4],hit[5],hit[6],hit[7],dphi,x,y])
            hit.append(dphi)
            hit.append(x)
            hit.append(y)
            #not using x and y
            hit[0] = r
            hit[1] = phi
            hitList.append(hit)
            
        if (len(hits) > maxhits):
            maxhits = len(hits)
            #print(hits)
            
        if(len(hits) >= 3):
            #tracks.append(hits[0:3])
            #print(hits[0:3])
            tracks.append(hits)

#store each track as an array of hits e.g [[x,y,z,t],[x,y,z,t],...]
print("number of rows is " + str(len(tracks)))
print("max hits is {}".format(maxhits))

In [ ]:
#generate a seed track
modulePositions = [-277.0, -252.0, -227.0, -202.0, -132.0, -62.0, -37.0, -12.0, 13.0, 
                   38.0, 63.0, 88.0, 113.0, 138.0, 163.0, 188.0, 213.0, 238.0, 263.0,
                   325.0, 402.0, 497.0, 616.0, 661.0, 706.0, 751.0]
maxZ = 751.0
minZ = -277.0
c = 3.0e8
psmm = 1.0e-9
sigma = 40

#direction (1 is right to left and -1 left to right)
def getNextzValue(currentZvalue,direction):
    zNext = 0
    if(direction == 1):
        if(currentZvalue == 751.0):
            zNext = 706.0
        elif(currentZvalue == 706.0):
            zNext = 661.0
        elif(currentZvalue == 661.0):
            zNext = 616.0
        elif(currentZvalue == 616.0):
            zNext = 497.0
        elif(currentZvalue == 497.0):
            zNext = 402.0
        elif(currentZvalue == 402.0):
            zNext = 325.0
        elif(currentZvalue == 325.0):
            zNext = 263.0
        elif(currentZvalue == -62.0):
            zNext = -132.0
        elif(currentZvalue == -132.0):
            zNext = -202.0
        else:
            zNext = currentZvalue - 25
    if(direction == -1):
        if(currentZvalue == -132.0):
            zNext = -62.0
        elif(currentZvalue == -202.0):
            zNext = -132.0
        else:
            zNext = currentZvalue + 25
    return zNext

def getNextz(track):
    zNext = 0
    direction = 0
    lastZ = track[-1][2]
    if(lastZ == 751.0):
        zNext = 706.0
    elif(track[-1][2] == 706.0):
        zNext = 661.0
    elif(track[-1][2] == 661.0):
        zNext = 616.0
    elif(track[-1][2] == 616.0):
        zNext = 497.0
    elif(track[-1][2] == 497.0):
        zNext = 402.0
    elif(track[-1][2] == 402.0):
        zNext = 325.0
    elif(track[-1][2] == 325.0):
        zNext = 263.0
    elif(lastZ == -62.0):
        zNext = -132.0
    elif(lastZ == -132.0):
        zNext = -202.0
    else:
        zNext = track[-1][2] - 25
    return zNext

#project the next hit assuming it is a line (with Gaussian scattering) and select a real hit which is closest
def predictR(track):
    z = []
    r = []
    for hit in track:
        z.append(hit[2])
        r.append(hit[0])
    #gradient,intercept
    results = np.polyfit(z,r,1,)
    zNext = 0
    zNext = getNextz(track)
    rNext = zNext*results[0] + results[1]
    #print(rNext)  
    return(rNext)
    
def predictPhi(track):
    z = []
    phi = []
    for hit in track:
        z.append(hit[2])
        phi.append(hit[1])
    #gradient,intercept
    results = np.polyfit(z,phi,1)
    zNext = getNextz(track)
    phiNext = zNext*results[0] + results[1]
    #print(phiNext) 
    return(phiNext)

def predictT(track):
    z = []
    times = []
    for hit in track:
        z.append(hit[2])
        times.append(hit[3])
    #gradient,intercept
    results = np.polyfit(z,times,1)
    zNext = getNextz(track)
    #deltaT = (track[-1][2] - zNext)/c/psmm
    #print(deltaT)
    #tNext = track[-1][2] - deltaT
    tNext = zNext*results[0] + results[1]
    #print(phiNext) 
    return(tNext)

In [ ]:
def checkCompatibility(hit1,hit2,direction):
    deltaZ = (hit2[2] - hit1[2])
    deltaT = (hit1[3] - hit2[3])
    tGap = abs(deltaZ/c/psmm)
    cond1 = False
    cond2 = False
    #print(tGap)
    if(deltaT > tGap - 3*hit1[7] and deltaT < tGap + 3*hit1[7] ):
        cond1 = True
        
    #check for phi consistency instead
    if(hit2[1] > hit1[1] - 5*hit1[8] and hit2[1] < hit1[1] + 5*hit1[8]):
        cond2 = True
    """
    theta = 0
    #cut on angle
    if(direction == 1):
        if(hit2[10] <= hit1[10]):
            cond2 = True
    elif(direction == -1):
        if(hit2[10] >= hit1[10]):
            cond1 = True
    """
    return (cond1 and cond2)


def getPartners(hit1,hitList,zVal,direction):
    candidates = []
    for hit in hitList:
        if(hit[2] != zVal):
            continue
        compatible = checkCompatibility(hit1,hit,direction)
        if(compatible):
            candidates.append(hit)
    return candidates


def fit(track):
    z = []
    phi = []
    for hit in track:
        z.append(hit[2])
        phi.append(hit[1])
    #gradient,intercept
    results = np.polyfit(z,phi,1)
    expected = []
    for val in z:
        expected.append(val*results[0] + results[1])
    chiSquared = np.sum((np.polyval(results,z) - phi)**2)
    return chiSquared
    
#should be closest in phi
def getClosestHit(prediction,hitList):
    closestHit = []
    closestDist = 1e6
    for hit in hitList:
        if(hit[2] != prediction[2]):
            continue
        """
        xPred = prediction[0]*np.cos(prediction[1])
        yPred = prediction[0]*np.sin(prediction[1])
        xHit = hit[9]
        yHit = hit[10]
        dist = np.sqrt((xPred - xHit)**2 + (yPred - yHit)**2)
        """
        dist = (hit[1] + np.pi) - (prediction[1] + np.pi)
        if(dist < closestDist):
            closestDist = dist
            closestHit = hit
    return closestHit

def fitSeed(seed,direction):
    #return a list of predictions for subsequent modules
    temp = seed.copy()
    
    predictions = []
    for i in range(0,5):
        predictedR = predictR(temp)
        predictedPhi = predictPhi(temp)
        predictedZ = getNextzValue(temp[-1][2],direction)
        predictedT = predictT(temp)
        predictedHit = [predictedR,predictedPhi,predictedZ,predictedT]
        temp.append(predictedHit)
    return temp

def buildTracks(hitList,startZ,modulePositions):
    tracks = []
    hit1List = []
    direction = 0
    if(startZ > 0.0):
        direction = 1
    elif(startZ < 0.0):
        direction = -1
    for hit in hitList:
        if(hit[2] == startZ):
            hit1List.append(hit)
    for hit1 in hit1List:
        #find compatible hits for this hit
        hit2List = getPartners(hit1,hitList,getNextzValue(startZ,direction),direction)
        #fit a straight line for the seed and predict the hits in each module
        predictedTracks = []
        for hit2 in hit2List:
            seed = [hit1,hit2]
            predictions = fitSeed(seed,direction)
            for i in range(2,len(predictions)):
                temp = getClosestHit(predictions[i],hitList)
                seed.append(temp)
            predictedTracks.append(seed)
        #fit the predictedTracks and choose one with best fit
        maxpVal = 0
        bestTrack = []
        for track in predictedTracks:
            chisquare = fit(track)
            pVal = stats.chi2.cdf(chisquare,len(track))
            if(pVal > maxpVal):
                maxpVal = pVal
                bestTrack = track
        tracks.append(bestTrack)
        #print(bestTrack)
                
    return tracks

In [ ]:
myTracks = buildTracks(hitList,751.0,modulePositions)

In [ ]:
print(myTracks[0])

In [ ]:
for track in tracks:
    for hit in track:
        if(hit[0] == myTracks[0][0][0]):
            print(track[::-1])
#plt.plot(zTrue,rTrue,marker='x',linestyle='None')
zPred = []
rPred = []
phiPred = []
for hit in myTracks[1]:
    zPred.append(hit[2])
    rPred.append(hit[0])
    phiPred.append(hit[1])
plt.plot(zPred,phiPred,marker='+',lineStyle='None',color='r')
plt.show()